In [30]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor,export_graphviz
import graphviz
import dtreeviz
import pandas as pd
import numpy as np
import pickle
import os
import json
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn import tree
from pypinyin import pinyin, lazy_pinyin, Style
from autogluon.tabular import TabularDataset, TabularPredictor

In [31]:
# autogluon保存路径
model_path = r"F:\cache_data\zone_ana\sb\modle\autogluon_type"
# 检查路径是否存在，否则便创建
if not os.path.exists(model_path):
    os.makedirs(model_path)
else:
    print("文件夹已存在")

文件夹已存在


In [32]:
dataset = pd.read_csv(r"F:\cache_data\zone_ana\sb\train_data\train_20240905.csv")

In [33]:
# 筛选满足条件的记录
pr = dataset[(dataset['MZMC'] == "河流冲积物") & (dataset['NEW_TZ'] == "中层砂质灰潮土")]

# 如果筛选出的记录少于 5 个，我们将修改所有记录
num_to_change = min(5, len(pr))

if num_to_change > 0:
    # 随机选择 num_to_change 个索引
    random_indices = np.random.choice(pr.index, num_to_change, replace=False)

    # 修改选中记录的 NEW_TZ 值
    dataset.loc[random_indices, 'NEW_TZ'] = "厚层砂质灰潮土"

    print(f"已随机修改 {num_to_change} 条记录的 NEW_TZ 值为 '厚层砂质灰潮土'")
else:
    print("没有找到满足条件的记录")

# 验证修改结果
modified_records = dataset[(dataset['MZMC'] == "河流冲积物") & (dataset['NEW_TZ'] == "厚层砂质灰潮土")]
print(f"修改后，符合新条件的记录数：{len(modified_records)}")

已随机修改 5 条记录的 NEW_TZ 值为 '厚层砂质灰潮土'
修改后，符合新条件的记录数：5


In [34]:
dataset.columns

Index(['OBJECTID', 'DLMC', 'DLDM', 'MZMC', 'NEW_TL', 'NEW_YL', 'NEW_TS',
       'NEW_TZ', 'Centroid_X', 'Centroid_Y',
       ...
       'slope_postion_101_smooth_MAX', 'slope_postion_101_smooth_RANGE',
       'slope_postion_101_smooth_MEAN', 'slope_postion_101_smooth_STD',
       'slope_postion_101_smooth_SUM', 'slope_postion_101_smooth_VARIETY',
       'slope_postion_101_smooth_MAJORITY',
       'slope_postion_101_smooth_MINORITY', 'slope_postion_101_smooth_MEDIAN',
       'slope_postion_101_smooth_PCT90'],
      dtype='object', length=420)

In [35]:
dataset["TZ_label"] = dataset.NEW_TZ.astype("category").cat.codes

In [36]:
dataset['DLMC'] = dataset['DLMC'].astype('category')
dataset['MZMC'] = dataset['MZMC'].astype('category')
dataset['TZ_label'] = dataset['TZ_label'].astype('category')

In [38]:
result = dataset.groupby('TZ_label', observed=True)["NEW_TZ"].apply(lambda x: list(x.unique())).to_dict()
for one_type in result:
    result[one_type] = result[one_type][0]
print(result)

{0: '中层壤质中性紫色土', 1: '中层壤质酸性紫色土', 2: '中层壤质黄色石灰土', 3: '中层暗泥质黄壤', 4: '中层泥质黄壤', 5: '中层灰泥质黄壤', 6: '中层砂泥质黄壤', 7: '中层砂质灰潮土', 8: '中层硅质黄壤', 9: '中层红泥质黄壤', 10: '中层黏质中性紫色土', 11: '中层黏质黄色石灰土', 12: '冷水泥田', 13: '厚层壤质中性紫色土', 14: '厚层壤质酸性紫色土', 15: '厚层壤质黄色石灰土', 16: '厚层泥质黄壤', 17: '厚层灰泥质黄壤', 18: '厚层砂泥质黄壤', 19: '厚层砂质灰潮土', 20: '厚层硅质黄壤', 21: '厚层红泥质黄壤', 22: '厚层黏质黄色石灰土', 23: '浅石灰泥田', 24: '浅紫泥田', 25: '浅鳝泥田', 26: '浅黄泥田', 27: '潮泥田', 28: '潮砂泥田', 29: '熟漂鳝泥田', 30: '石灰泥田', 31: '石鳝泥田', 32: '紫泥田', 33: '腐中层壤质中性紫色土', 34: '腐中层壤质酸性紫色土', 35: '腐中层壤质黄色石灰土', 36: '腐中层暗泥质黄壤', 37: '腐中层泥质黄壤', 38: '腐中层泥质黄棕壤', 39: '腐中层灰泥质黄壤', 40: '腐中层砂泥质黄壤', 41: '腐中层硅质黄壤', 42: '腐中层红泥质黄壤', 43: '腐中层黏质中性紫色土', 44: '腐中层黏质黄色石灰土', 45: '腐厚层壤质中性紫色土', 46: '腐厚层壤质酸性紫色土', 47: '腐厚层壤质黄色石灰土', 48: '腐厚层暗泥质黄壤', 49: '腐厚层泥质黄壤', 50: '腐厚层泥质黄棕壤', 51: '腐厚层灰泥质黄壤', 52: '腐厚层砂泥质黄壤', 53: '腐厚层硅质黄壤', 54: '腐厚层红泥质黄壤', 55: '腐厚层黏质黄色石灰土', 56: '腐薄层壤质中性紫色土', 57: '腐薄层壤质酸性紫色土', 58: '腐薄层壤质黄色石灰土', 59: '腐薄层暗泥质黄壤', 60: '腐薄层泥质酸性粗骨土', 61: '腐薄层泥质黄壤', 62: '腐薄层泥质黄棕壤', 63: '腐薄层灰泥质石灰性粗骨土', 64: '腐薄层灰泥质黄

In [39]:
# 保存映射关系
with open(r'D:\worker_code\Terrain_Test\data\soil_dict_20240905_sb.json', 'w') as f:
    json.dump(result, f)

In [40]:
# 查看地类类别
dl_df = pd.DataFrame(dataset['DLMC'].value_counts())

In [41]:
dl_df

,count
DLMC,
乔木林地,3469
旱地,1248
灌木林地,1060
水田,1022
天然牧草地,86
果园,77
其他林地,39
其他草地,20
设施农用地,12


In [42]:
# 查看母质类别
mz_df = pd.DataFrame(dataset['MZMC'].value_counts())

In [43]:
mz_df

,count
MZMC,
碳酸盐岩,3792
砂岩,922
泥(页)岩,862
第四纪红色粘土,568
紫红色砂(页)岩,344
河流冲积物,209
砂页岩,163
凝灰岩,161
板岩,30


### 砂岩土种

In [44]:
def delete_row(df, mzmc, new_tz):
    query_result = df[(df['MZMC'] == f'{mzmc}') & (df['NEW_TZ'] == f'{new_tz}')]
    df = df.drop(query_result.index)
    print(query_result.shape[0])
    return df


In [46]:
dataset = delete_row(dataset,"砂岩","石灰泥田")
dataset = delete_row(dataset,"砂岩","黄青砂泥田")


1
4


In [47]:
# 筛选砂岩水稻土数据
sy_sdt_data = dataset[dataset['DLMC'].isin(['水田','水浇地','坑塘水面','养殖坑塘','内陆滩涂']) 
                      & (dataset['MZMC'] == '砂岩')]
pd.unique(sy_sdt_data['NEW_TZ']),pd.unique(sy_sdt_data['NEW_TS'])

(array(['黄浅白粉泥田', '黄白粉泥田', '轻漂白粉泥田', '重漂白粉泥田', '黄青白粉泥田', '黄黄白粉泥田'],
       dtype=object),
 array(['浅白粉泥田', '白粉泥田', '漂白粉泥田', '青白粉泥田', '黄白粉泥田'], dtype=object))

In [49]:
dataset = delete_row(dataset,"砂岩","中层壤质酸性紫色土")

1


In [50]:
# 筛砂岩选非自然土数据
sy_fzrt_data = dataset[~dataset['DLMC'].isin(['乔木林地','灌木林地','竹林地','其他林地','其他草地','天然牧草地','人工牧草地','水田','水浇地','坑塘水面','养殖坑塘','内陆滩涂']) 
                       & (dataset['MZMC'] == '砂岩')]
pd.unique(sy_fzrt_data['NEW_TZ']),pd.unique(sy_fzrt_data['NEW_TS'])

(array(['中层硅质黄壤', '厚层硅质黄壤', '薄层硅质黄壤'], dtype=object),
 array(['硅质黄壤'], dtype=object))

In [51]:
# 筛选自然土数据
sy_zrt_data = dataset[dataset['DLMC'].isin(['乔木林地','灌木林地','竹林地','其他林地','其他草地','天然牧草地','人工牧草地'])
                   & (dataset['MZMC'] == '砂岩')]
pd.unique(sy_zrt_data['NEW_TZ']),pd.unique(sy_zrt_data['NEW_TS'])

(array(['腐薄层硅质黄壤', '腐厚层硅质黄壤', '腐中层硅质黄壤', '腐薄层硅质酸性粗骨土'], dtype=object),
 array(['硅质黄壤', '硅质酸性粗骨土'], dtype=object))

### 碳酸岩土种

In [53]:
dataset = delete_row(dataset,"碳酸盐岩","青灰泥田")

47


In [54]:
# 筛选碳酸岩水稻土数据
tsy_sdt_data = dataset[dataset['DLMC'].isin(['水田','水浇地','坑塘水面','养殖坑塘','内陆滩涂']) 
                      & (dataset['MZMC'] == '碳酸盐岩')]
pd.unique(tsy_sdt_data['NEW_TZ']),pd.unique(tsy_sdt_data['NEW_TS']),

(array(['浅石灰泥田', '黄浅灰泥田', '石灰泥田', '青石灰泥田', '冷水泥田'], dtype=object),
 array(['浅灰泥田', '灰泥田', '青灰泥田', '冷水泥田'], dtype=object))

In [55]:
# 筛选碳酸岩选非自然土数据
# tsy_fzrt_data = dataset[~dataset['DLMC'].isin(['乔木林地','灌木林地','竹林地','其他林地','其他草地','天然牧草地','人工牧草地','水田','水浇地','坑塘水面','养殖坑塘','内陆滩涂']) 
#                        & ( dataset['NEW_TZ'].str.contains('灰')) & (dataset['MZMC'] != '泥(页)岩') &(dataset['MZMC'] != '第四系红粘土')]
tsy_fzrt_data = dataset[~dataset['DLMC'].isin(['乔木林地','灌木林地','竹林地','其他林地','其他草地','天然牧草地','人工牧草地','水田','水浇地','坑塘水面','养殖坑塘','内陆滩涂']) 
                       & ( dataset['MZMC']=='碳酸盐岩')]

pd.unique(tsy_fzrt_data['NEW_TZ']),pd.unique(tsy_fzrt_data['NEW_TS']),

(array(['中层壤质黄色石灰土', '中层黏质黄色石灰土', '厚层黏质黄色石灰土', '厚层灰泥质黄壤', '薄层灰泥质石灰性粗骨土',
        '中层灰泥质黄壤', '薄层灰泥质黄壤', '薄层壤质黄色石灰土', '厚层壤质黄色石灰土', '薄层黏质黄色石灰土'],
       dtype=object),
 array(['壤质黄色石灰土', '黏质黄色石灰土', '灰泥质黄壤', '灰泥质石灰性粗骨土'], dtype=object))

In [56]:
# 筛选碳酸岩自然土数据
# tsy_zrt_data = dataset[dataset['DLMC'].isin(['乔木林地','灌木林地','竹林地','其他林地','其他草地','天然牧草地','人工牧草地'])
#                    & ( dataset['NEW_TZ'].str.contains('灰')) & (dataset['MZMC'] != '泥(页)岩') &(dataset['MZMC'] != '第四纪红色粘土')]
tsy_zrt_data = dataset[dataset['DLMC'].isin(['乔木林地','灌木林地','竹林地','其他林地','其他草地','天然牧草地','人工牧草地'])
                   &(( dataset['MZMC']=='碳酸盐岩')&(dataset['NEW_TZ'].str.contains('灰')))]
pd.unique(tsy_zrt_data['NEW_TZ']),pd.unique(tsy_zrt_data['NEW_TS']),

(array(['腐中层壤质黄色石灰土', '腐薄层壤质黄色石灰土', '腐厚层壤质黄色石灰土', '腐中层灰泥质黄壤', '腐中层黏质黄色石灰土',
        '腐厚层黏质黄色石灰土', '腐厚层灰泥质黄壤', '腐薄层灰泥质黄壤', '腐薄层灰泥质石灰性粗骨土', '腐薄层黏质黄色石灰土'],
       dtype=object),
 array(['壤质黄色石灰土', '灰泥质黄壤', '黏质黄色石灰土', '灰泥质石灰性粗骨土'], dtype=object))

### 第四系红粘土

In [57]:
# 筛选第四系红粘土水稻土数据
hnt_sdt_data = dataset[dataset['DLMC'].isin(['水田','水浇地','坑塘水面','养殖坑塘','内陆滩涂']) 
                      & (dataset['MZMC'] == '第四纪红色粘土')]
pd.unique(hnt_sdt_data['NEW_TZ']),pd.unique(hnt_sdt_data['NEW_TS']),

(array(['黄浅红泥田', '黄黄泥田', '黄红泥田', '黄青红泥田'], dtype=object),
 array(['浅红泥田', '黄泥田', '红泥田', '青红泥田'], dtype=object))

In [59]:
# 筛选第四系红粘土非自然土数据
hnt_fzrt_data = dataset[~dataset['DLMC'].isin(['乔木林地','灌木林地','竹林地','其他林地','其他草地','天然牧草地','人工牧草地','水田','水浇地','坑塘水面','养殖坑塘','内陆滩涂']) 
                       & ((dataset['MZMC'] == '第四纪红色粘土')&(dataset['NEW_TZ'].str.contains('红')))]
pd.unique(hnt_fzrt_data['NEW_TZ']),pd.unique(hnt_fzrt_data['NEW_TS']),

(array(['中层红泥质黄壤', '厚层红泥质黄壤', '薄层红泥质黄壤'], dtype=object),
 array(['红泥质黄壤'], dtype=object))

In [60]:
# 筛选第四系红粘土自然土数据
hnt_zrt_data = dataset[dataset['DLMC'].isin(['乔木林地','灌木林地','竹林地','其他林地','其他草地','天然牧草地','人工牧草地'])
                   & (dataset['MZMC'] == '第四纪红色粘土')]
pd.unique(hnt_zrt_data['NEW_TZ']),pd.unique(hnt_zrt_data['NEW_TS']),

(array(['腐中层红泥质黄壤', '腐薄层壤质黄色石灰土', '腐中层黏质黄色石灰土', '腐厚层壤质黄色石灰土', '腐中层壤质黄色石灰土',
        '腐厚层红泥质黄壤', '腐薄层红泥质黄壤', '腐厚层黏质黄色石灰土'], dtype=object),
 array(['红泥质黄壤', '壤质黄色石灰土', '黏质黄色石灰土'], dtype=object))

### 泥(页)岩

In [63]:
dataset = delete_row(dataset,"泥(页)岩","青鳝泥田")
dataset = delete_row(dataset,"泥(页)岩","浅鳝泥田")
dataset = delete_row(dataset,"泥(页)岩","熟漂鳝泥田")
dataset = delete_row(dataset,"泥(页)岩","石鳝泥田")
dataset = delete_row(dataset,"泥(页)岩","潮砂泥田")
dataset = delete_row(dataset,"泥(页)岩","黄红泥田")
dataset = delete_row(dataset,"泥(页)岩","潮泥田")

0
49
11
12
3
5
1


In [64]:
# 筛选泥(页)岩水稻土数据
nyy_sdt_data = dataset[dataset['DLMC'].isin(['水田','水浇地','坑塘水面','养殖坑塘','内陆滩涂']) 
                      & (dataset['MZMC'] == '泥(页)岩')]
pd.unique(nyy_sdt_data['NEW_TZ']),pd.unique(nyy_sdt_data['NEW_TS']),

(array(['黄浅鳝泥田', '黄鳝泥田', '黄黄泥田', '浅黄泥田', '轻漂鳝泥田'], dtype=object),
 array(['浅鳝泥田', '鳝泥田', '黄泥田', '浅黄泥田', '漂鳝泥田'], dtype=object))

In [65]:
# 筛选泥(页)岩非自然土数据
nyy_fzrt_data = dataset[~dataset['DLMC'].isin(['乔木林地','灌木林地','竹林地','其他林地','其他草地','天然牧草地','人工牧草地','水田','水浇地','坑塘水面','养殖坑塘','内陆滩涂']) 
                       & (dataset['MZMC'] == '泥(页)岩')]
pd.unique(nyy_fzrt_data['NEW_TZ']),pd.unique(nyy_fzrt_data['NEW_TS']),

(array(['厚层泥质黄壤', '中层泥质黄壤', '薄层泥质黄壤', '薄层泥质酸性粗骨土'], dtype=object),
 array(['泥质黄壤', '泥质酸性粗骨土'], dtype=object))

In [66]:
dataset = delete_row(dataset,"泥(页)岩","腐中层灰泥质黄壤")

1


In [67]:
# 筛选泥(页)岩自然土数据
nyy_zrt_data = dataset[dataset['DLMC'].isin(['乔木林地','灌木林地','竹林地','其他林地','其他草地','天然牧草地','人工牧草地'])
                   & (dataset['MZMC'] == '泥(页)岩')]
pd.unique(nyy_zrt_data['NEW_TZ']),pd.unique(nyy_zrt_data['NEW_TZ']),

(array(['腐中层泥质黄壤', '腐厚层泥质黄壤', '腐薄层泥质黄壤', '腐厚层泥质黄棕壤', '腐薄层泥质酸性粗骨土',
        '腐中层泥质黄棕壤'], dtype=object),
 array(['腐中层泥质黄壤', '腐厚层泥质黄壤', '腐薄层泥质黄壤', '腐厚层泥质黄棕壤', '腐薄层泥质酸性粗骨土',
        '腐中层泥质黄棕壤'], dtype=object))

### 紫红色砂页岩	

In [68]:
# 筛选紫红色砂页岩水稻土数据
zhsyy_sdt_data = dataset[dataset['DLMC'].isin(['水田','水浇地','坑塘水面','养殖坑塘','内陆滩涂']) 
                      & ((dataset['MZMC'] == '紫红色砂(页)岩'))]
pd.unique(zhsyy_sdt_data['NEW_TZ']),pd.unique(zhsyy_sdt_data['NEW_TS']),

(array(['酸浅紫泥田', '紫泥田', '青紫泥田', '浅紫泥田'], dtype=object),
 array(['浅紫泥田', '紫泥田', '青紫泥田'], dtype=object))

In [69]:
# 筛选紫红色砂页岩非自然土数据
zhsyy_fzrt_data = dataset[~dataset['DLMC'].isin(['乔木林地','灌木林地','竹林地','其他林地','其他草地','天然牧草地','人工牧草地','水田','水浇地','坑塘水面','养殖坑塘','内陆滩涂']) 
                       & (dataset['MZMC'] == '紫红色砂(页)岩')]
pd.unique(zhsyy_fzrt_data['NEW_TZ']),pd.unique(zhsyy_fzrt_data['NEW_TS']),

(array(['厚层壤质酸性紫色土', '中层壤质中性紫色土', '厚层壤质中性紫色土', '中层黏质中性紫色土', '薄层壤质中性紫色土',
        '中层壤质酸性紫色土', '薄层壤质酸性紫色土'], dtype=object),
 array(['壤质酸性紫色土', '壤质中性紫色土', '黏质中性紫色土'], dtype=object))

In [70]:
# 筛选紫红色砂页岩自然土数据
zhsyy_zrt_data = dataset[dataset['DLMC'].isin(['乔木林地','灌木林地','竹林地','其他林地','其他草地','天然牧草地','人工牧草地'])
                   & (dataset['MZMC'] == '紫红色砂(页)岩')]
pd.unique(zhsyy_zrt_data['NEW_TZ']),pd.unique(zhsyy_zrt_data['NEW_TS']),

(array(['腐厚层壤质酸性紫色土', '腐中层壤质中性紫色土', '腐厚层壤质中性紫色土', '腐薄层壤质中性紫色土',
        '腐薄层壤质酸性紫色土', '腐中层黏质中性紫色土', '腐中层壤质酸性紫色土'], dtype=object),
 array(['壤质酸性紫色土', '壤质中性紫色土', '黏质中性紫色土'], dtype=object))

### 河流冲积物

In [71]:
# 筛选河流冲积物水稻土数据
hlcjw_sdt_data = dataset[dataset['DLMC'].isin(['水田','水浇地','坑塘水面','养殖坑塘','内陆滩涂']) &
                         (dataset['NEW_TZ'].str.contains('潮')&(dataset['MZMC'] == '河流冲积物'))]
pd.unique(hlcjw_sdt_data['NEW_TZ']),pd.unique(hlcjw_sdt_data['NEW_TS']),

(array(['轻漂潮泥田', '潮砂泥田', '潮泥田', '青潮泥田', '重漂潮泥田', '黄潮泥田'], dtype=object),
 array(['漂潮泥田', '潮砂泥田', '潮泥田', '青潮泥田', '黄潮泥田'], dtype=object))

In [72]:
# 筛选河流冲积物非自然土数据
hlcjw_fzrt_data = dataset[~dataset['DLMC'].isin(['乔木林地','灌木林地','竹林地','其他林地','其他草地','天然牧草地','人工牧草地','水田','水浇地','坑塘水面','养殖坑塘','内陆滩涂']) &
                         (dataset['NEW_TZ'].str.contains('潮')&(dataset['MZMC'] == '河流冲积物'))]

pd.unique(hlcjw_fzrt_data['NEW_TZ']),pd.unique(hlcjw_fzrt_data['NEW_TS']),

(array(['厚层砂质灰潮土', '中层砂质灰潮土'], dtype=object), array(['砂质灰潮土'], dtype=object))

### 板岩

In [73]:
# 筛选板岩水稻土数据
by_sdt_data = dataset[dataset['DLMC'].isin(['水田','水浇地','坑塘水面','养殖坑塘','内陆滩涂']) 
                      & ((dataset['MZMC'] == '板岩'))]
pd.unique(by_sdt_data['NEW_TZ']),pd.unique(by_sdt_data['NEW_TS']),

(array([], dtype=object), array([], dtype=object))

In [74]:
# 筛选板岩非自然土数据
by_fzrt_data = dataset[~dataset['DLMC'].isin(['乔木林地','灌木林地','竹林地','其他林地','其他草地','天然牧草地','人工牧草地','水田','水浇地','坑塘水面','养殖坑塘','内陆滩涂']) 
                       & ((dataset['MZMC'] == '板岩'))]
pd.unique(by_fzrt_data['NEW_TZ']),pd.unique(by_fzrt_data['NEW_TZ']),

(array([], dtype=object), array([], dtype=object))

In [75]:
# 筛选板岩非自然土数据
by_zrt_data = dataset[dataset['DLMC'].isin(['乔木林地','灌木林地','竹林地','其他林地','其他草地','天然牧草地','人工牧草地']) 
                       & ((dataset['MZMC'] == '板岩'))]
pd.unique(by_zrt_data['NEW_TZ']),pd.unique(by_zrt_data['NEW_TS']),

(array(['腐薄层泥质黄棕壤', '腐中层泥质黄棕壤', '腐厚层泥质黄棕壤'], dtype=object),
 array(['泥质暗黄棕壤'], dtype=object))

### 凝灰岩

In [76]:
# 筛选凝灰岩水稻土数据
nhy_sdt_data = dataset[dataset['DLMC'].isin(['水田','水浇地','坑塘水面','养殖坑塘','内陆滩涂']) 
                       & ((dataset['MZMC'] == '凝灰岩'))
                     ]
pd.unique(nhy_sdt_data['NEW_TZ']),pd.unique(nhy_sdt_data['NEW_TS'])

(array(['黄浅暗泥田', '黄暗泥田'], dtype=object), array(['浅暗泥田', '暗泥田'], dtype=object))

In [77]:
# 筛选凝灰岩非自然土数据
nhy_fzrt_data = dataset[~dataset['DLMC'].isin(['乔木林地','灌木林地','竹林地','其他林地','其他草地','天然牧草地','人工牧草地','水田','水浇地','坑塘水面','养殖坑塘','内陆滩涂']) 
                       & ((dataset['MZMC'] == '凝灰岩'))]
pd.unique(nhy_fzrt_data['NEW_TZ']),pd.unique(nhy_fzrt_data['NEW_TS']),

(array(['中层暗泥质黄壤', '薄层暗泥质黄壤'], dtype=object), array(['暗泥质黄壤'], dtype=object))

In [78]:
# 筛选凝灰岩自然土数据
nhy_zrt_data = dataset[dataset['DLMC'].isin(['乔木林地','灌木林地','竹林地','其他林地','其他草地','天然牧草地','人工牧草地'])
                   & (dataset['MZMC'] == '凝灰岩')]
pd.unique(nhy_zrt_data['NEW_TZ']),pd.unique(nhy_zrt_data['NEW_TS']),

(array(['腐中层暗泥质黄壤', '腐薄层暗泥质黄壤', '腐厚层暗泥质黄壤'], dtype=object),
 array(['暗泥质黄壤'], dtype=object))

### 砂页岩

In [79]:
# 筛选砂页岩水稻土数据
syy_sdt_data = dataset[dataset['DLMC'].isin(['水田','水浇地','坑塘水面','养殖坑塘','内陆滩涂']) 
                      & (dataset['MZMC'] == '砂页岩')]
pd.unique(syy_sdt_data['NEW_TZ']),pd.unique(syy_sdt_data['NEW_TS']),

(array(['黄青砂泥田', '黄浅砂泥田', '黄砂泥田', '重漂砂泥田', '轻漂砂泥田'], dtype=object),
 array(['青砂泥田', '浅砂泥田', '砂泥田', '漂砂泥田'], dtype=object))

In [80]:
# 筛选砂页岩非自然土数据
syy_fzrt_data = dataset[~dataset['DLMC'].isin(['乔木林地','灌木林地','竹林地','其他林地','其他草地','天然牧草地','人工牧草地','水田','水浇地','坑塘水面','养殖坑塘','内陆滩涂']) 
                       & (dataset['MZMC'] == '砂页岩')]
pd.unique(syy_fzrt_data['NEW_TZ']),pd.unique(syy_fzrt_data['NEW_TS']),

(array(['厚层砂泥质黄壤', '中层砂泥质黄壤'], dtype=object), array(['砂泥质黄壤'], dtype=object))

In [81]:
# 筛选砂页岩物自然土数据
syy_zrt_data = dataset[dataset['DLMC'].isin(['乔木林地','灌木林地','竹林地','其他林地','其他草地','天然牧草地','人工牧草地'])
                   & (dataset['MZMC'] == '砂页岩')]
pd.unique(syy_zrt_data['NEW_TZ']),pd.unique(syy_zrt_data['NEW_TS']),

(array(['腐厚层砂泥质黄壤', '腐中层砂泥质黄壤', '腐薄层砂泥质黄壤'], dtype=object),
 array(['砂泥质黄壤'], dtype=object))

In [150]:
columns_List = [_ for _ in dataset.columns]

In [151]:
columns_List

['OBJECTID',
 'DLMC',
 'DLDM',
 'MZMC',
 'NEW_TL',
 'NEW_YL',
 'NEW_TS',
 'NEW_TZ',
 'Centroid_X',
 'Centroid_Y',
 'aligned_AnalyticalHillshading_COUNT',
 'aligned_AnalyticalHillshading_AREA',
 'aligned_AnalyticalHillshading_MIN',
 'aligned_AnalyticalHillshading_MAX',
 'aligned_AnalyticalHillshading_RANGE',
 'aligned_AnalyticalHillshading_MEAN',
 'aligned_AnalyticalHillshading_STD',
 'aligned_AnalyticalHillshading_SUM',
 'aligned_AnalyticalHillshading_MEDIAN',
 'aligned_AnalyticalHillshading_PCT90',
 'aligned_ChannelNetworkBaseLevel_COUNT',
 'aligned_ChannelNetworkBaseLevel_AREA',
 'aligned_ChannelNetworkBaseLevel_MIN',
 'aligned_ChannelNetworkBaseLevel_MAX',
 'aligned_ChannelNetworkBaseLevel_RANGE',
 'aligned_ChannelNetworkBaseLevel_MEAN',
 'aligned_ChannelNetworkBaseLevel_STD',
 'aligned_ChannelNetworkBaseLevel_SUM',
 'aligned_ChannelNetworkBaseLevel_MEDIAN',
 'aligned_ChannelNetworkBaseLevel_PCT90',
 'aligned_ChannelNetworkDistance_COUNT',
 'aligned_ChannelNetworkDistance_AREA',
 'a

In [152]:
# 定义要检查的关键字列表
keywords = ['MEAN']

# 定义包含字符串的列表
strings = list(dataset.columns)

# 使用列表推导式和any函数检查并添加到新列表
filtered_strings = [s for s in strings if any(keyword in s for keyword in keywords)]
print(len(filtered_strings))
print(filtered_strings)


41
['aligned_AnalyticalHillshading_MEAN', 'aligned_ChannelNetworkBaseLevel_MEAN', 'aligned_ChannelNetworkDistance_MEAN', 'aligned_ConvergenceIndex_MEAN', 'aligned_Analytical_Hillshading_MEAN', 'aligned_Aspect_MEAN', 'aligned_Channel_Network_Base_Level_MEAN', 'aligned_Channel_Network_Distance_MEAN', 'aligned_Convergence_Index_MEAN', 'aligned_dem_MEAN', 'aligned_ETP2022_3_MEAN', 'aligned_ETP2022_8_MEAN', 'aligned_ETP2022_mean_MEAN', 'aligned_evi_MEAN', 'aligned_LS_Factor_MEAN', 'aligned_lswi_MEAN', 'aligned_mndwi_MEAN', 'aligned_ndmi_MEAN', 'aligned_ndvi_MEAN', 'aligned_ndwi_MEAN', 'aligned_NIGHT2022_MEAN', 'aligned_pca_1_MEAN', 'aligned_pca_2_MEAN', 'aligned_Plan_Curvature_MEAN', 'aligned_PRE2022_3_MEAN', 'aligned_PRE2022_8_MEAN', 'aligned_PRE2022_mean_MEAN', 'aligned_Profile_Curvature_MEAN', 'aligned_Relative_Slope_Position_MEAN', 'aligned_savi_MEAN', 'aligned_Slope_MEAN', 'aligned_TMP2022_3_MEAN', 'aligned_TMP2022_8_MEAN', 'aligned_TMP2022_mean_MEAN', 'aligned_Topographic_Wetness_Inde

In [82]:

sdt_features= ['DLMC','MZMC','Centroid_X','Centroid_Y','aligned_AnalyticalHillshading_MEAN','aligned_ChannelNetworkBaseLevel_MEAN','aligned_ChannelNetworkDistance_MEAN','aligned_ConvergenceIndex_MEAN',
 'aligned_Analytical_Hillshading_MEAN','aligned_Aspect_MEAN','aligned_Channel_Network_Base_Level_MEAN','aligned_Channel_Network_Distance_MEAN','aligned_Convergence_Index_MEAN',
 'aligned_dem_MEAN','aligned_ETP2022_3_MEAN','aligned_ETP2022_8_MEAN','aligned_ETP2022_mean_MEAN','aligned_evi_MEAN','aligned_LS_Factor_MEAN','aligned_lswi_MEAN','aligned_mndwi_MEAN',
 'aligned_ndmi_MEAN','aligned_ndvi_MEAN','aligned_ndwi_MEAN','aligned_NIGHT2022_MEAN','aligned_pca_1_MEAN','aligned_pca_2_MEAN','aligned_Plan_Curvature_MEAN','aligned_PRE2022_3_MEAN',
 'aligned_PRE2022_8_MEAN','aligned_PRE2022_mean_MEAN','aligned_Profile_Curvature_MEAN','aligned_Relative_Slope_Position_MEAN','aligned_savi_MEAN','aligned_Slope_MEAN','aligned_TMP2022_3_MEAN',
 'aligned_TMP2022_8_MEAN','aligned_TMP2022_mean_MEAN','aligned_Topographic_Wetness_Index_MEAN','aligned_Total_Catchment_Area_MEAN','aligned_Valley_Depth_MEAN','aligned_vari_MEAN',
 'MRRTF_MEAN','MRVBF_MEAN','slope_postion_101_smooth_MAJORITY']
fzrt_features = ['DLMC','MZMC','Centroid_X','Centroid_Y','aligned_AnalyticalHillshading_MEAN','aligned_ChannelNetworkBaseLevel_MEAN','aligned_ChannelNetworkDistance_MEAN','aligned_ConvergenceIndex_MEAN',
 'aligned_Analytical_Hillshading_MEAN','aligned_Aspect_MEAN','aligned_Channel_Network_Base_Level_MEAN','aligned_Channel_Network_Distance_MEAN','aligned_Convergence_Index_MEAN',
 'aligned_dem_MEAN','aligned_ETP2022_3_MEAN','aligned_ETP2022_8_MEAN','aligned_ETP2022_mean_MEAN','aligned_evi_MEAN','aligned_LS_Factor_MEAN','aligned_lswi_MEAN','aligned_mndwi_MEAN',
 'aligned_ndmi_MEAN','aligned_ndvi_MEAN','aligned_ndwi_MEAN','aligned_NIGHT2022_MEAN','aligned_pca_1_MEAN','aligned_pca_2_MEAN','aligned_Plan_Curvature_MEAN','aligned_PRE2022_3_MEAN',
 'aligned_PRE2022_8_MEAN','aligned_PRE2022_mean_MEAN','aligned_Profile_Curvature_MEAN','aligned_Relative_Slope_Position_MEAN','aligned_savi_MEAN','aligned_Slope_MEAN','aligned_TMP2022_3_MEAN',
 'aligned_TMP2022_8_MEAN','aligned_TMP2022_mean_MEAN','aligned_Topographic_Wetness_Index_MEAN','aligned_Total_Catchment_Area_MEAN','aligned_Valley_Depth_MEAN','aligned_vari_MEAN',
 'MRRTF_MEAN','MRVBF_MEAN','slope_postion_101_smooth_MAJORITY']
zrt_features = ['DLMC','MZMC','Centroid_X','Centroid_Y','aligned_AnalyticalHillshading_MEAN','aligned_ChannelNetworkBaseLevel_MEAN','aligned_ChannelNetworkDistance_MEAN','aligned_ConvergenceIndex_MEAN',
 'aligned_Analytical_Hillshading_MEAN','aligned_Aspect_MEAN','aligned_Channel_Network_Base_Level_MEAN','aligned_Channel_Network_Distance_MEAN','aligned_Convergence_Index_MEAN',
 'aligned_dem_MEAN','aligned_ETP2022_3_MEAN','aligned_ETP2022_8_MEAN','aligned_ETP2022_mean_MEAN','aligned_evi_MEAN','aligned_LS_Factor_MEAN','aligned_lswi_MEAN','aligned_mndwi_MEAN',
 'aligned_ndmi_MEAN','aligned_ndvi_MEAN','aligned_ndwi_MEAN','aligned_NIGHT2022_MEAN','aligned_pca_1_MEAN','aligned_pca_2_MEAN','aligned_Plan_Curvature_MEAN','aligned_PRE2022_3_MEAN',
 'aligned_PRE2022_8_MEAN','aligned_PRE2022_mean_MEAN','aligned_Profile_Curvature_MEAN','aligned_Relative_Slope_Position_MEAN','aligned_savi_MEAN','aligned_Slope_MEAN','aligned_TMP2022_3_MEAN',
 'aligned_TMP2022_8_MEAN','aligned_TMP2022_mean_MEAN','aligned_Topographic_Wetness_Index_MEAN','aligned_Total_Catchment_Area_MEAN','aligned_Valley_Depth_MEAN','aligned_vari_MEAN',
 'MRRTF_MEAN','MRVBF_MEAN','slope_postion_101_smooth_MAJORITY']
target = "TZ_label"

### 砂岩数据集

In [83]:
# 获取sdt数据集
sy_sdt_data = sy_sdt_data[sdt_features+[target]]
# 获取非自然土数据集
sy_fzrt_data = sy_fzrt_data[fzrt_features+[target]]
# 获取自然土数据集
sy_zrt_data = sy_zrt_data[zrt_features+[target]]

In [84]:
sy_sdt_data.columns,len(sy_sdt_data.columns)

(Index(['DLMC', 'MZMC', 'Centroid_X', 'Centroid_Y',
        'aligned_AnalyticalHillshading_MEAN',
        'aligned_ChannelNetworkBaseLevel_MEAN',
        'aligned_ChannelNetworkDistance_MEAN', 'aligned_ConvergenceIndex_MEAN',
        'aligned_Analytical_Hillshading_MEAN', 'aligned_Aspect_MEAN',
        'aligned_Channel_Network_Base_Level_MEAN',
        'aligned_Channel_Network_Distance_MEAN',
        'aligned_Convergence_Index_MEAN', 'aligned_dem_MEAN',
        'aligned_ETP2022_3_MEAN', 'aligned_ETP2022_8_MEAN',
        'aligned_ETP2022_mean_MEAN', 'aligned_evi_MEAN',
        'aligned_LS_Factor_MEAN', 'aligned_lswi_MEAN', 'aligned_mndwi_MEAN',
        'aligned_ndmi_MEAN', 'aligned_ndvi_MEAN', 'aligned_ndwi_MEAN',
        'aligned_NIGHT2022_MEAN', 'aligned_pca_1_MEAN', 'aligned_pca_2_MEAN',
        'aligned_Plan_Curvature_MEAN', 'aligned_PRE2022_3_MEAN',
        'aligned_PRE2022_8_MEAN', 'aligned_PRE2022_mean_MEAN',
        'aligned_Profile_Curvature_MEAN',
        'aligned_Relative_Slo

### 碳酸岩数据集

In [85]:
# 获取sdt数据集
tsy_sdt_data = tsy_sdt_data[sdt_features+[target]]
# 获取非自然土数据集
tsy_fzrt_data = tsy_fzrt_data[fzrt_features+[target]]
# 获取自然土数据集
tsy_zrt_data = tsy_zrt_data[zrt_features+[target]]

### 第四系红粘土数据集

In [86]:
# 获取sdt数据集
hnt_sdt_data = hnt_sdt_data[sdt_features+[target]]
# 获取非自然土数据集
hnt_fzrt_data = hnt_fzrt_data[fzrt_features+[target]]
# 获取自然土数据集
hnt_zrt_data = hnt_zrt_data[zrt_features+[target]]

### 泥(页)岩数据集

In [87]:
# 获取sdt数据集
nyy_sdt_data = nyy_sdt_data[sdt_features+[target]]
# 获取非自然土数据集
nyy_fzrt_data = nyy_fzrt_data[fzrt_features+[target]]
# 获取自然土数据集
nyy_zrt_data = nyy_zrt_data[zrt_features+[target]]

### 紫红色砂页岩数据集

In [88]:
# 获取sdt数据集
zhsyy_sdt_data = zhsyy_sdt_data[sdt_features+[target]]
# 获取非自然土数据集
zhsyy_fzrt_data = zhsyy_fzrt_data[fzrt_features+[target]]
# 获取自然土数据集
zhsyy_zrt_data = zhsyy_zrt_data[zrt_features+[target]]

### 河流冲积物数据集

In [89]:
# 获取sdt数据集
hlcjw_sdt_data = hlcjw_sdt_data[sdt_features+[target]]
# 获取非自然土
hlcjw_fzrt_data = hlcjw_fzrt_data[sdt_features+[target]]
# 获取非自然土

### 板岩数据集

In [90]:
# 获取自然土数据集
by_zrt_data = by_zrt_data[zrt_features+[target]]

### 凝灰岩数据集

In [91]:
# 获取sdt数据集
nhy_sdt_data = nhy_sdt_data[sdt_features+[target]]
# 获取非自然土数据集
nhy_fzrt_data = nhy_fzrt_data[fzrt_features+[target]]
# 获取自然土数据集
nhy_zrt_data = nhy_zrt_data[zrt_features+[target]]

### 砂页岩数据集

In [92]:
# 获取sdt数据集
syy_sdt_data = syy_sdt_data[sdt_features+[target]]
# 获取非自然土数据集
syy_fzrt_data = syy_fzrt_data[fzrt_features+[target]]
# 获取自然土数据集
syy_zrt_data = syy_zrt_data[zrt_features+[target]]

In [93]:
# 查看各个数据集的大小
print('砂岩',sy_sdt_data.shape,sy_fzrt_data.shape,sy_zrt_data.shape)
print('碳酸岩',tsy_sdt_data.shape,tsy_fzrt_data.shape,tsy_zrt_data.shape)
print('第四系红粘土',hnt_sdt_data.shape,hnt_fzrt_data.shape,hnt_zrt_data.shape)
print('泥页岩',nyy_sdt_data.shape,nyy_fzrt_data.shape,nyy_zrt_data.shape)
print('紫红色砂页岩',zhsyy_sdt_data.shape,zhsyy_fzrt_data.shape,zhsyy_zrt_data.shape)
print('河流冲积物',hlcjw_sdt_data.shape,hlcjw_fzrt_data.shape)
print('板岩',by_zrt_data.shape)
print('凝灰岩',nhy_sdt_data.shape,nhy_fzrt_data.shape,nhy_zrt_data.shape)
print('砂页岩',syy_sdt_data.shape,syy_fzrt_data.shape,syy_zrt_data.shape)


砂岩 (115, 46) (141, 46) (660, 46)
碳酸岩 (179, 46) (640, 46) (2917, 46)
第四系红粘土 (47, 46) (107, 46) (268, 46)
泥页岩 (136, 46) (140, 46) (489, 46)
紫红色砂页岩 (93, 46) (107, 46) (144, 46)
河流冲积物 (158, 46) (10, 46)
板岩 (30, 46)
凝灰岩 (31, 46) (22, 46) (108, 46)
砂页岩 (74, 46) (41, 46) (48, 46)


In [94]:
# 确定标签
label = "TZ_label"
eval_metric = 'f1_weighted'
problem_type = 'multiclass'
# 初始化模型和超参数
hyperparameters={
	'NN_TORCH': {},
	'FASTAI': {},
	'RF': [{'criterion': 'gini', 'ag_args': {'name_suffix': 'Gini', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'entropy', 'ag_args': {'name_suffix': 'Entr', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'squared_error', 'ag_args': {'name_suffix': 'MSE', 'problem_types': ['regression', 'quantile']}}],
	'XT': [{'criterion': 'gini', 'ag_args': {'name_suffix': 'Gini', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'entropy', 'ag_args': {'name_suffix': 'Entr', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'squared_error', 'ag_args': {'name_suffix': 'MSE', 'problem_types': ['regression', 'quantile']}}]
}


### 模型训练

In [95]:
def train_model(train_data,label,eval_metric,hyperparameters,problem_type,model_master,model_branch,model_root):
    # 训练model
    temp_master_path =os.path.join(model_root,model_master)
    temp_branch_path = os.path.join(temp_master_path,model_branch)
    # 检查路径是否存在，否则便创建
    if not os.path.exists(temp_branch_path):
        os.makedirs(temp_branch_path)
    else:
        print("文件夹已存在")
    # 执行训练
    train_predictor = TabularPredictor(label=label,path=temp_branch_path,eval_metric=eval_metric,problem_type=problem_type).fit(train_data,hyperparameters=hyperparameters)
    train_predictor.fit_summary()
    return train_predictor.model_best


### 模型训练

In [96]:
# 砂岩
sy_sdt_predictor = train_model(sy_sdt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='sy',model_branch='sdt',model_root=model_path) 
sy_fzrt_predictor = train_model(sy_fzrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='sy',model_branch='fzrt',model_root=model_path) 
sy_zrt_predictor = train_model(sy_zrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='sy',model_branch='zrt',model_root=model_path) 
# 碳酸岩
tsy_sdt_predictor = train_model(tsy_sdt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='tsy',model_branch='sdt',model_root=model_path) 
tsy_fzrt_predictor = train_model(tsy_fzrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='tsy',model_branch='fzrt',model_root=model_path) 
tsy_zrt_predictor = train_model(tsy_zrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='tsy',model_branch='zrt',model_root=model_path) 
# 第四系红粘土
hnt_sdt_predictor = train_model(hnt_sdt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='hnt',model_branch='sdt',model_root=model_path) 
hnt_fzrt_predictor = train_model(hnt_fzrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='hnt',model_branch='fzrt',model_root=model_path) 
hnt_zrt_predictor = train_model(hnt_zrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='hnt',model_branch='zrt',model_root=model_path) 
# 泥页岩
nyy_sdt_predictor = train_model(nyy_sdt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='nyy',model_branch='sdt',model_root=model_path) 
nyy_fzrt_predictor = train_model(nyy_fzrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='nyy',model_branch='fzrt',model_root=model_path) 
nyy_zrt_predictor = train_model(nyy_zrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='nyy',model_branch='zrt',model_root=model_path) 
# 紫红色砂页岩
zhsyy_sdt_predictor = train_model(zhsyy_sdt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='zhsyy',model_branch='sdt',model_root=model_path) 
zhsyy_fzrt_predictor = train_model(zhsyy_fzrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='zhsyy',model_branch='fzrt',model_root=model_path) 
zhsyy_zrt_predictor = train_model(zhsyy_zrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='zhsyy',model_branch='zrt',model_root=model_path) 
# 河流冲积物
hlcjw_sdt_predictor = train_model(hlcjw_sdt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='hlcjw',model_branch='sdt',model_root=model_path)
hlcjw_fzrt_predictor = train_model(hlcjw_fzrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='hlcjw',model_branch='fzrt',model_root=model_path)

# 板岩
ly_zrt_predictor = train_model(by_zrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='by',model_branch='zrt',model_root=model_path) 
# 凝灰岩
nhy_sdt_predictor = train_model(nhy_sdt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='nhy',model_branch='sdt',model_root=model_path) 
nhy_fzrt_predictor = train_model(nhy_fzrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='nhy',model_branch='fzrt',model_root=model_path) 
nhy_zrt_predictor = train_model(nhy_zrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='nhy',model_branch='zrt',model_root=model_path) 
# 砂页岩
syy_sdt_predictor = train_model(syy_sdt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='syy',model_branch='sdt',model_root=model_path) 
syy_fzrt_predictor = train_model(syy_fzrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='syy',model_branch='fzrt',model_root=model_path) 
syy_zrt_predictor = train_model(syy_zrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='syy',model_branch='zrt',model_root=model_path) 

No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ...
AutoGluon will save models to "F:\cache_data\zone_ana\sb\modle\autogluon_type\sy\sdt"
=================== System Info ===================
AutoGluon Version:  1.1.0
Python Version:     3.10.11
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22631
CPU Count:          20
Memory Avail:       47.89 GB / 63.81 GB (75.0%)
Disk Space Avail:   277.69 GB / 

*** Summary of fit() ***
Estimated performance of each model:
                 model  score_val  eval_metric  pred_time_val  fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0  WeightedEnsemble_L2   0.569829  f1_weighted       0.035342  2.559003                0.000000           0.080739            2       True          7
1       ExtraTreesEntr   0.523300  f1_weighted       0.034312  0.412757                0.034312           0.412757            1       True          5
2       NeuralNetTorch   0.496537  f1_weighted       0.000000  1.401021                0.000000           1.401021            1       True          6
3      NeuralNetFastAI   0.461620  f1_weighted       0.001030  2.065507                0.001030           2.065507            1       True          1
4       ExtraTreesGini   0.437405  f1_weighted       0.033361  0.416229                0.033361           0.416229            1       True          4
5     RandomForestGini   0.408148  f1_

No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ...
AutoGluon will save models to "F:\cache_data\zone_ana\sb\modle\autogluon_type\sy\fzrt"
=================== System Info ===================
AutoGluon Version:  1.1.0
Python Version:     3.10.11
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22631
CPU Count:          20
Memory Avail:       47.77 GB / 63.81 GB (74.9%)
Disk Space Avail:   277.69 GB /

Plot summary of models saved to file: F:\cache_data\zone_ana\sb\modle\autogluon_type\sy\sdtSummaryOfModels.html
*** End of fit() summary ***


No improvement since epoch 3: early stopping
	0.5206	 = Validation score   (f1_weighted)
	0.43s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.4569	 = Validation score   (f1_weighted)
	0.33s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.5189	 = Validation score   (f1_weighted)
	0.33s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.4503	 = Validation score   (f1_weighted)
	0.33s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.4233	 = Validation score   (f1_weighted)
	0.33s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: NeuralNetTorch ...
	0.621	 = Validation score   (f1_weighted)
	0.75s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	Ensemble Weights: {'NeuralNetTorch': 0.667, 'NeuralNetFastAI': 0.333}
	0.6515	 = Validation score   (f1_weighted)
	0.08s	 = Training

...

*** Summary of fit() ***
Estimated performance of each model:
                 model  score_val  eval_metric  pred_time_val  fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0  WeightedEnsemble_L2   0.686247  f1_weighted       0.000000  0.532296                0.000000           0.187505            2       True          7
1      NeuralNetFastAI   0.686247  f1_weighted       0.000000  0.344790                0.000000           0.344790            1       True          1
2       ExtraTreesGini   0.675556  f1_weighted       0.031189  0.375231                0.031189           0.375231            1       True          4
3       ExtraTreesEntr   0.675556  f1_weighted       0.031190  0.391169                0.031190           0.391169            1       True          5
4     RandomForestEntr   0.675556  f1_weighted       0.031208  0.485431                0.031208           0.485431            1       True          3
5     RandomForestGini   0.675556  f1_

	0.7381	 = Validation score   (f1_weighted)
	0.36s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.5333	 = Validation score   (f1_weighted)
	0.45s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.4762	 = Validation score   (f1_weighted)
	0.52s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.4762	 = Validation score   (f1_weighted)
	0.42s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
No improvement since epoch 2: early stopping
	0.7833	 = Validation score   (f1_weighted)
	0.2s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: NeuralNetTorch ...
	0.5333	 = Validation score   (f1_weighted)
	0.17s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	Ensemble Weights: {'NeuralNetFastAI': 1.0}
	0.7833	 = Validation score   (f1_weighted)
	0.11s	 = Training   runtime
	0.0s	 = Validat

*** Summary of fit() ***
Estimated performance of each model:
                 model  score_val  eval_metric  pred_time_val  fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0  WeightedEnsemble_L2   0.783333  f1_weighted       0.015630  0.313337                0.000000           0.109364            2       True          7
1      NeuralNetFastAI   0.783333  f1_weighted       0.015630  0.203974                0.015630           0.203974            1       True          5
2     RandomForestGini   0.738095  f1_weighted       0.048772  0.359996                0.048772           0.359996            1       True          1
3       NeuralNetTorch   0.533333  f1_weighted       0.000000  0.172265                0.000000           0.172265            1       True          6
4     RandomForestEntr   0.533333  f1_weighted       0.031250  0.453185                0.031250           0.453185            1       True          2
5       ExtraTreesEntr   0.476190  f1_

No improvement since epoch 4: early stopping
	0.7083	 = Validation score   (f1_weighted)
	0.22s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.6659	 = Validation score   (f1_weighted)
	0.33s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.6659	 = Validation score   (f1_weighted)
	0.31s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.615	 = Validation score   (f1_weighted)
	0.33s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.615	 = Validation score   (f1_weighted)
	0.36s	 = Training   runtime
	0.06s	 = Validation runtime
Fitting model: NeuralNetTorch ...
	0.6659	 = Validation score   (f1_weighted)
	0.19s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	Ensemble Weights: {'NeuralNetFastAI': 1.0}
	0.7083	 = Validation score   (f1_weighted)
	0.13s	 = Training   runtime
	0.0s	 = Validati

*** Summary of fit() ***
Estimated performance of each model:
                 model  score_val  eval_metric  pred_time_val  fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0  WeightedEnsemble_L2   0.708333  f1_weighted       0.000000  0.343716                0.000000           0.125031            2       True          7
1      NeuralNetFastAI   0.708333  f1_weighted       0.000000  0.218685                0.000000           0.218685            1       True          1
2       NeuralNetTorch   0.665909  f1_weighted       0.000000  0.188484                0.000000           0.188484            1       True          6
3     RandomForestEntr   0.665909  f1_weighted       0.015563  0.312561                0.015563           0.312561            1       True          3
4     RandomForestGini   0.665909  f1_weighted       0.031190  0.328822                0.031190           0.328822            1       True          2
5       ExtraTreesGini   0.615000  f1_